In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-10-18 17:39:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-10-18 17:39:26 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SportsReviewVine").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 52.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a4938788ecb6f8a0b6d41083161bca3d44044b1029690679e1a8181da4478d6e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

print(url.split("/")[-1])
df = spark.read.option('header', 'true').csv(SparkFiles.get(url.split("/")[-1]), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()


amazon_reviews_us_Sports_v1_00.tsv.gz
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-01-31 00:08:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG

In [ ]:
#Retreiving the dataset and count the vine reviews
dataset = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [ ]:
# Retrieve all the rows where the total_votes count is equal to or greater than 20
vine_df = dataset.filter("total_votes >= 20")

In [ ]:
#Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
votes_greater_equals_50 = vine_df.filter(vine_df["helpful_votes"]/vine_df["total_votes"]>=0.5)

In [ ]:
#Determine the total number of reviews,
votes_greater_equals_50.count()

61948

In [ ]:

#DataFrame of reviews written as part of the Vine program (paid), vine == 'Y'.
vine_yes_df = votes_greater_equals_50.filter(votes_greater_equals_50["vine"] == "Y")
#DataFrame of reviews written as part of the Vine progra (unpaid), vine == 'N'.
vine_no_df = votes_greater_equals_50.filter(votes_greater_equals_50["vine"] == "N")


In [ ]:
#Count of the non-vine reviews
vine_no_df.count()

61614

In [ ]:
#Count of the vine reviews
vine_yes_df.count()

334

In [ ]:
#All rows with a rating of 5
five_star = votes_greater_equals_50.filter(votes_greater_equals_50["star_rating"]== 5)
five_star.count()

32804

In [30]:
#Count of the vine reviews
five_start_yes = five_star.filter(five_star["vine"] == "Y").count()
five_start_yes

139

In [31]:
#Count of the non-vine reviews
five_star_no = five_star.filter(five_star["vine"] == "N").count()
five_star_no

32665

In [32]:
#Percentage of 5-star paid reviews
five_start_yes/votes_greater_equals_50.filter(votes_greater_equals_50["vine"]== "Y").count()

0.4161676646706587

In [33]:
#Percentage of 5-star unpaid reviews
five_star_no/votes_greater_equals_50.filter(votes_greater_equals_50["vine"]== "N").count()

0.5301554841432142

# **Summary**




*   It appears after analyzing the data that there is a small diffrence of 11.4% between the vine reviews (41.61%) and the non-vine ones (53.01%) which represents a slight bias of them in the vine program.
*   In addition to this analyse we could calculate the mean, median and mode of the dataset for the Vine and non-Vine reviews.

